In [4]:
import numpy as np
import matplotlib as plt
import pandas as pd
import random
%matplotlib inline
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

import requests
import json
import re

In [5]:
# url_path = "https://www.land.mlit.go.jp/webland/api/TradeListSearch?from=20171&to=20184&area=13"
# request_result = requests.get(url_path)
# data_json = request_result.json()["data"]
# print(len(data_json))
# print(data_json[0])

In [6]:
# data_pd = pd.io.json.json_normalize(data_json)
# print(data_pd.shape)
# print(data_pd.head(10))
# https://www.land.mlit.go.jp/webland/api.html
# https://www.land.mlit.go.jp/webland/download.html

In [7]:
data_from_csv = pd.read_csv("data/13_Tokyo_20171_20184.csv", encoding='cp932')
# print(data_from_csv.shape)
# print(data_from_csv.iloc[0])
# print(data_from_csv.head(10))
# print(data_from_csv["種類"].unique())

In [8]:
data_used_apartment = data_from_csv.query('種類 == "中古マンション等"')
# print(data_used_apartment.shape)
# print(data_used_apartment.isnull().sum())

In [14]:
columns_name_list = ["最寄駅：距離（分）", "間取り", "面積（㎡）", "建築年", "建物の構造", "建ぺい率（％）", "容積率（％）", "市区町村名", "取引価格（総額）"]
data_selected = data_used_apartment[columns_name_list]
print(data_selected.shape)

data_selected_dropna = data_selected.dropna(how='any') # 1つでもNaNデータを含む行を削除
print(data_selected_dropna.shape)
print(data_selected_dropna.iloc[0])

(30956, 9)
(28442, 9)
最寄駅：距離（分）           3
間取り              １ＬＤＫ
面積（㎡）              35
建築年             昭和60年
建物の構造             ＳＲＣ
建ぺい率（％）            80
容積率（％）            700
市区町村名            千代田区
取引価格（総額）     24000000
Name: 0, dtype: object


In [22]:
# print(data_selected_dropna["建築年"].unique())
data_selected_dropna = data_selected_dropna[data_selected_dropna["建築年"].str.match('^平成|昭和')]
# print(data_selected_dropna["建築年"].unique())

wareki_to_seireki = {'昭和': 1926-1, '平成': 1989-1}

building_year_list = data_selected_dropna["建築年"]

building_age_list = []
for building_year in building_year_list:
    # 昭和x年 -> 昭和, xに変換、平成x年 -> 平成, xに変換
    building_year_split = re.search(r'(.+?)([0-9]+|元)年', building_year)
    # 西暦に変換
    seireki = wareki_to_seireki[building_year_split.groups()[0]] + int(building_year_split.groups()[1])

    building_age = 2019 - seireki
    building_age_list.append(building_age)

data_selected_dropna["築年数"] = building_age_list
data_added_building_age = data_selected_dropna.drop("建築年", axis=1)
# print(data_added_building_age.head())

In [25]:
columns_name_list = ["最寄駅：距離（分）", "面積（㎡）", "築年数", "建ぺい率（％）", "容積率（％）", "取引価格（総額）"]
dummy_list = ["間取り", "建物の構造", "市区町村名"]

data_added_dummies = pd.concat([data_added_building_age[columns_name_list], pd.get_dummies(data_added_building_age[dummy_list], drop_first=True)], axis=1)

print(data_added_dummies.shape)
print(data_added_dummies.iloc[0])

(28438, 87)
最寄駅：距離（分）               3
面積（㎡）                  35
築年数                    34
建ぺい率（％）                80
容積率（％）                700
取引価格（総額）         24000000
間取り_スタジオ                0
間取り_１ＤＫ                 0
間取り_１ＤＫ＋Ｓ               0
間取り_１Ｋ                  0
間取り_１Ｋ＋Ｓ                0
間取り_１ＬＤＫ                1
間取り_１ＬＤＫ＋Ｓ              0
間取り_１Ｒ                  0
間取り_２ＤＫ                 0
間取り_２ＤＫ＋Ｓ               0
間取り_２Ｋ                  0
間取り_２Ｋ＋Ｓ                0
間取り_２ＬＤＫ                0
間取り_２ＬＤＫ＋Ｓ              0
間取り_３ＤＫ                 0
間取り_３ＤＫ＋Ｓ               0
間取り_３Ｋ                  0
間取り_３ＬＤＫ                0
間取り_３ＬＤＫ＋Ｋ              0
間取り_３ＬＤＫ＋Ｓ              0
間取り_３ＬＫ                 0
間取り_４ＤＫ                 0
間取り_４Ｋ                  0
間取り_４ＬＤＫ                0
                   ...   
市区町村名_日野市               0
市区町村名_昭島市               0
市区町村名_杉並区               0
市区町村名_東久留米市             0
市区町村名_東大和市              0
市区町村名_東村山市              0
市区町村名_板橋区               0
